<a href="https://colab.research.google.com/github/indl1670/KST_Project/blob/main/1cycle_sideproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/kst_project/oneCycle

/content/drive/MyDrive/kst_project/oneCycle


In [ ]:
git clone https://github.com/ultralytics/yolov5.git

In [3]:
cd yolov5

/content/drive/MyDrive/kst_project/oneCycle/yolov5


In [ ]:
!pip install -r requirements.txt
!pip install wandb

In [ ]:
import os

dataset_dir = '/content/drive/Shareddrives/KST Project #2/split_image'
dataset_folder = os.listdir(dataset_dir)

# 수량 일치 확인
for dataset in dataset_folder:
  print(str(dataset))
  img, lab = os.listdir(os.path.join(dataset_dir, dataset))
  if lab == 'images': # 폴더 순서 설정
    lab = 'labels'
    img = 'images'
    
  img_list = os.listdir(os.path.join(dataset_dir, dataset, img))
  lab_list = os.listdir(os.path.join(dataset_dir, dataset, lab))
  cnt=0
  for l in lab_list:
    l2i = l[:-3] + 'jpg'
    if l2i in img_list:
      cnt+=1
  print('img: ', len(img_list))
  print('lab: ', len(lab_list))
  print('match: ', cnt)


train
img:  987
lab:  987
match:  987
valid
img:  253
lab:  253
match:  253
test
img:  170
lab:  170
match:  170


In [ ]:
!python train.py --img 640 --batch 16 --epochs 100 --data ./data/custom.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt

wandb: Currently logged in as: heoseokyong (use `wandb login --relogin` to force relogin)
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=./data/custom.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 8 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 2021-12-7 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0

In [ ]:
!python detect.py --weights ./runs/train/exp12/weights/best.pt --img 640 --conf 0.5 --source /content/drive/Shareddrives/KST_Project/split_image2/test/images --save-txt

detect: weights=['./runs/train/exp12/weights/best.pt'], source=/content/drive/Shareddrives/KST_Project/split_image2/test/images, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-124-g1075488 torch 1.10.0+cu111 CPU

Fusing layers... 
Model Summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/170 /content/drive/Shareddrives/KST_Project/split_image2/test/images/AM_sunny_CI02_20211101_100015_839_3.jpg: 384x640 8 road_holes, Done. (0.257s)
image 2/170 /content/drive/Shareddrives/KST_Project/split_image2/test/images/AM_sunny_CI02_20211101_100015_840_2.jpg: 384x640 8 road_holes, Done. (0.271s)
image 3/170 /content/drive/Shareddrives/KST_Project/split_image2/

In [32]:
import cv2
import os
import numpy as np
import shutil

def drawline(img,pt1,pt2,color,thickness=1,style='dotted',gap=20):
    dist =((pt1[0]-pt2[0])**2+(pt1[1]-pt2[1])**2)**.5
    pts= []
    for i in  np.arange(0,dist,gap):
        r=i/dist
        x=int((pt1[0]*(1-r)+pt2[0]*r)+.5)
        y=int((pt1[1]*(1-r)+pt2[1]*r)+.5)
        p = (x,y)
        pts.append(p)

    if style=='dotted':
        for p in pts:
            cv2.circle(img,p,thickness,color,-1)
    else:
        s=pts[0]
        e=pts[0]
        i=0
        for p in pts:
            s=e
            e=p
            if i%2==1:
                cv2.line(img,s,e,color,thickness)
            i+=1

def drawpoly(img,pts,color,thickness=1,style='dotted',):
    s=pts[0]
    e=pts[0]
    pts.append(pts.pop(0))
    for p in pts:
        s=e
        e=p
        drawline(img,s,e,color,thickness,style)
def drawrect(img,pt1,pt2,color,thickness=1,style='dotted'):
    pts = [pt1,(pt2[0],pt1[1]),pt2,(pt1[0],pt2[1])] 
    drawpoly(img,pts,color,thickness,style)

dir = '/content/drive/MyDrive/kst_project/oneCycle/yolov5/runs/detect/exp2'
test_label_dir = '/content/drive/Shareddrives/KST_Project/split_image2/test/labels'

test_labels = os.listdir(test_label_dir)

if os.path.isdir('/content/drive/MyDrive/kst_project/oneCycle/yolov5/runs/detect/exp2/TEST/'):
  shutil.rmtree('/content/drive/MyDrive/kst_project/oneCycle/yolov5/runs/detect/exp2/TEST/')

os.mkdir('/content/drive/MyDrive/kst_project/oneCycle/yolov5/runs/detect/exp2/TEST/')

for label in test_labels:
  file_name = label[:-4]
  img = cv2.imread(os.path.join(dir, file_name + '.jpg'))

  with open(os.path.join(test_label_dir, label), 'r') as f:
    a = f.read()
    line = a.split('\n')

    for  li in line:
      if li == '':
        continue
      c, x, y, w, h = li.split(' ')
      x = float(x) * 1920
      y = float(y) * 1080
      w = float(w) * 1920
      h = float(h) * 1080
      drawrect(img, (round(x-(w/2)), round(y-(h/2))), (round(x+(w/2)), round(y+(h/2))),(255,0,0), 3, 'dotted')
  cv2.imwrite('/content/drive/MyDrive/kst_project/oneCycle/yolov5/runs/detect/exp2/TEST/' + file_name + '.jpg', img)

